# Le Taquin

## Génération des problèmes

In [2]:
from math import sqrt
def getNbProblems():
    while(1):
        try: 
            m = int(input("Veuillez saisir un nombre de grille à générer : "))
            break
        except:
            print("Vous n'avez pas saisi une valeur numérique.")
    return m
            
def getLenProblem():
    while(1):
        try:             
            n = int(input("Veuillez saisir un entier n, strictement supérieur à 1, tel que la grille soit de taille n*n : "))
            if n > 1 :
                break
            
        except:
            print("Vous n'avez pas saisi une valeur numérique")
    return n

def printGrid(grid):
    n = sqrt(len(grid))
    grid_str = ""
    for i in range(len(grid)):
        if (i%n == 0):
            grid_str += "\n | "
        grid_str += str(grid[i]) + " | "
    print(grid_str+"\n\n")            

In [3]:
# init = [1,2,3,4,5,6,7,8,0] pour n = 3
def init(n):
    init = list(range(1,n*n))
    init.append(0)
    return init

def gauche(pb, pos):
    n = int(sqrt(len(pb)))
    if not(pos%n == 0) :
        pb[pos] = pb[pos-1]
        pb[pos-1] = 0
        pos = pos-1
    return (pb, pos)

def droite(pb, pos):
    n = int(sqrt(len(pb)))
    if not(pos%n == n-1) :
        pb[pos] = pb[pos+1]
        pb[pos+1] = 0
        pos = pos+1
    return (pb, pos)

def haut(pb, pos):
    n = int(sqrt(len(pb)))
    if not(pos < n) :
        pb[pos] = pb[pos-n]
        pb[pos-n] = 0
        pos  = pos-n
    return (pb, pos)

def bas(pb, pos):
    n = int(sqrt(len(pb)))
    if not(pos >= (n*n)-n) :
        pb[pos] = pb[pos+n]
        pb[pos+n] = 0
        pos = pos+n
    return (pb, pos)

def moveHole(dep, pb, pos):
    # [0:gauche,1:droite,2:haut,3:bas]
    if dep == 0:
        return gauche(pb, pos)
    if dep == 1:
        return droite(pb, pos)
    if dep == 2:
        return haut(pb, pos)
    if dep == 3:
        return bas(pb, pos)

In [4]:
import random

# init = [1,2,3,4,5,6,7,8,0] pour n = 3
def genProblem(init, n):
    pb = init
    pos = (n*n)-1
    # Le nb de déplacements pourrait varier en fonction de la difficultée choisie (amélioration à apporter)
    nb_deplacements = random.randint(15*n, 40*n)
    for i in range(nb_deplacements):
        # [0:gauche,1:droite,2:haut,3:bas]
        list_dep = [0,1,2,3]
        if pos < n :
            list_dep.remove(2)
        if pos >= n*n-n :
            list_dep.remove(3)        
        if pos%n == 0 :
            list_dep.remove(0)
        if pos%n == n-1 :
            list_dep.remove(1)
        (pb, pos) = moveHole(random.choice(list_dep), pb, pos)
    return pb

In [5]:
def genProblems(m,n):
    problems = []
    for i in range(m):
            problems.append(genProblem(init(n),n))
    return problems

In [6]:
def genFile():
    m = getNbProblems()
    n = getLenProblem()
    name_file = str(m)+"puzzle"+str(n)+"x"+str(n)+"problems"
    f = open(name_file,"w+")
    l_pbs = genProblems(m, n)
    f.write("Dimensions : "+str(n)+"x"+str(n)+"\n")
    for pb in l_pbs:
        f.write(str(pb)+"\n")
    print("\nLes jeux de taquin ont étés créées dans le fichier "+name_file+"\n")
    f.close() 


## Implentation de BFS

In [7]:
def getChemin(noeud):
    return noeud[1]

def getEtat(noeud):
    return noeud[0]

def final(taquin):
    final = list(range(1,len(taquin)))
    final.append(0)
    if taquin == final :
        return True
    return False

def getChemin(noeud):
    return noeud[1]

def getEtat(noeud):
    return noeud[0]

def dejavu(etat,chemin):
    for noeud in chemin:
        if etat==getEtat(noeud):
            return True
    return False

def actions():
    return [gauche, droite, haut, bas]

def noeud_init():
    while(True):                
        try: 
            file = input("\nVeuillez saisir le nom du fichier : ")
            f = open(file, "r")
            break
        except:
            print("\nFichier introuvable.\n")           
    
    grille = []
    n = int(f.readlines()[0].split('x')[1])
    f.close()
    f = open(file, "r")
    for l in f.readlines():
        grille.append(list((int(c) for c in l if c.isdigit())))
    f.close()
    grille = grille[1:]
    
    while(True):
        try:
            num_line = int(input("Veuillez saisir le numéro du jeu à résoudre (indices à partir de 0) : "))
            printGrid(grille[num_line])
            return (grille[num_line],[],0,0)
        except:
            print("\nNuméro de ligne invalide\n")

# CF Implémentations de init() et des actions dans la partie génération

In [8]:
def manhattanTile(tile, posTile, n):
    colPos = posTile % n
    lnPos = posTile // n
    colVal = (tile-1) % n
    lnVal = (tile-1) // n
    col = abs(colPos - colVal)
    ln = abs(lnPos - lnVal)
    distance = col + ln
    return distance

def manhattanGrid(grid,n):
    somme = 0
    for i in range(len(grid)):
        if grid[i] == 0:
            somme += manhattanTile(n*n,n*n-1,n)
        else:
            somme += manhattanTile(grid[i],i,n)  
    return somme

def nbTilesMalPlacees(grid, n):
    nb = 0
    for i in range(n*n):
        if i != (n*n-1) :            
            if grid[i] != i+1:
                nb += 1
        else:
            if grid[i] != 0:
                nb += 1
    return nb

def heuristiques():
    return [manhattanGrid, nbTilesMalPlacees]

def coutAction():
    return 1

def noeudsFilsTaquin(noeud, nbH):
    (etatCourant,cheminCourant,g,heur)=noeud
    listeFils=[]
    pos = etatCourant.index(0)
    coutAct = coutAction()
    n = int(sqrt(len(etatCourant)))
    for action in actions():
        etatTest = etatCourant.copy()
        (etatSuiv,p) = action(etatTest,pos)
        if (not dejavu(etatSuiv,cheminCourant)) and etatSuiv != etatCourant :
            chemin=list(cheminCourant)
            chemin.append(noeud)
            listeFils.append((etatSuiv,chemin,g+coutAct,heuristiques()[nbH](etatSuiv,n)))
    return listeFils

In [20]:
def choixHeuristique():
    while(True):
        nbH = input("""Choisissez une heuristique pour la résolution du jeu :
        (0) Distances de Manhattan
        (1) Nombre de tuiles mal placées\n\n""")
        if(nbH == "0" or nbH == "1"):
            nbH = int(nbH)
            break
    return nbH

def afficher_sol(noeud):
    print("Solution :")
    chemin=getChemin(noeud)
    for i in chemin:
        printGrid(getEtat(i))
    printGrid(getEtat(noeud))
    print("Nombre de coups : " + str(len(chemin)+1))

In [21]:
def bfsTaquin(noeud):
    listSuiv=[noeud]
    nbH = choixHeuristique()
    while(1):
        firstElem = listSuiv[0]
        if final(getEtat(firstElem)):
            return (True,firstElem)
        else:
            filsSuiv = noeudsFilsTaquin(firstElem, nbH)
            listSuiv += filsSuiv
            listSuiv = listSuiv[1:]
            if (listSuiv == []):
                return False
            listSuiv.sort(key=lambda elt:elt[3], reverse=False)

## Implémentation de A*

In [22]:
def AStarTaquin(noeud):
    listSuiv=[noeud]
    nbH = choixHeuristique()
    while(1):
        firstElem = listSuiv[0]
        if final(getEtat(firstElem)):
            return (True,firstElem)
        else:
            filsSuiv = noeudsFilsTaquin(firstElem, nbH)
            listSuiv += filsSuiv
            listSuiv = listSuiv[1:]
            if (listSuiv == []):
                return False
            listSuiv.sort(key=lambda elt:elt[2]+elt[3], reverse=False)

# Résolvez des grilles de taquin

In [23]:
def taquin():
    while(True):
        mode = input("""Que souhaitez-vous faire :
        (0) Quitter 
        (1) Générer des grilles de jeu 
        (2) Résoudre une grille d'un fichier\n\n""")
        if(mode == "1"):
            genFile()
        elif(mode == "2"):
            noeud = noeud_init()
            print("Résolution avec l'approche BFS : ")
            afficher_sol(bfsTaquin(noeud)[1])
            print("Résolution avec l'approche A* : ")
            afficher_sol(AStarTaquin(noeud)[1])
            
        elif(mode == "0"):
            break
        else:
            print("\nVeuillez saisir 0, 1 ou 2\n")
            pass
        
    

In [24]:
taquin()

Que souhaitez-vous faire :
        (0) Quitter 
        (1) Générer des grilles de jeu 
        (2) Résoudre une grille d'un fichier

2

Veuillez saisir le nom du fichier : 9puzzle3x3problems
Veuillez saisir le numéro du jeu à résoudre (indices à partir de 0) : 0

 | 7 | 1 | 4 | 
 | 8 | 0 | 6 | 
 | 2 | 5 | 3 | 


Résolution avec l'approche BFS : 
Choisissez une heuristique pour la résolution du jeu :
        (0) Distances de Manhattan
        (1) Nombre de tuiles mal placées

0
Solution :

 | 7 | 1 | 4 | 
 | 8 | 0 | 6 | 
 | 2 | 5 | 3 | 



 | 7 | 1 | 4 | 
 | 0 | 8 | 6 | 
 | 2 | 5 | 3 | 



 | 0 | 1 | 4 | 
 | 7 | 8 | 6 | 
 | 2 | 5 | 3 | 



 | 1 | 0 | 4 | 
 | 7 | 8 | 6 | 
 | 2 | 5 | 3 | 



 | 1 | 4 | 0 | 
 | 7 | 8 | 6 | 
 | 2 | 5 | 3 | 



 | 1 | 4 | 6 | 
 | 7 | 8 | 0 | 
 | 2 | 5 | 3 | 



 | 1 | 4 | 6 | 
 | 7 | 8 | 3 | 
 | 2 | 5 | 0 | 



 | 1 | 4 | 6 | 
 | 7 | 8 | 3 | 
 | 2 | 0 | 5 | 



 | 1 | 4 | 6 | 
 | 7 | 8 | 3 | 
 | 0 | 2 | 5 | 



 | 1 | 4 | 6 | 
 | 0 | 8 | 3 | 
 | 7 | 2 | 5 |